In [2]:
import numpy as np
import xarray as xr
import pandas as pd 
import sys 
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading US biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:228: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(
/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:613: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(
/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:228: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(


constructing catch trawl


In [3]:
# %%time
# # code to turn length_US file into a dataframe with expanded frequencies
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
# df = epro_2019.process_length_data_df(out, 0)
# df.head()

In [4]:
out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
df = epro_2019.process_length_data_ds_wu_jung(out, 0)

/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:316: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(


In [5]:
df.head()

,Haul,Sex,Length,Frequency
0,1,2,20.0,5
1,1,2,21.0,13
2,1,2,22.0,38
3,1,2,23.0,38
4,1,2,24.0,16


In [6]:
df.Length.unique()

array([20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32.,
       33., 18., 19.,  7.,  9., 51., 34., 39., 35., 37., 40., 41., 42.,
       43., 44., 45., 48., 52., 54., 55., 61., 38., 47., 36., 46., 49.,
       50., 57., 60., 62., 58., 59., 63., 64., 70., 53., 65., 67., 69.,
       56., 71., 66., 72., 74., 68., 73., 77.])

In [7]:
haul_len_multi = pd.MultiIndex.from_product([list(df.Haul.unique()), list(df.Length.unique()), [1, 2]], names=['Haul', 'Length', 'Sex'])

In [8]:
df.set_index(['Haul', 'Length', 'Sex'], inplace=True)
# df.reindex(haul_len_multi)

In [31]:
# df.index.value_counts()
temp = df.groupby(level=[0, 1, 2]).sum().to_xarray()

In [58]:
temp['n'] = temp.Frequency.sum(['Length', 'Sex'])

In [61]:
temp['meanlen'] = (temp.Length*temp.Frequency).sum(['Length', 'Sex'])/ temp.n

In [69]:
temp['stdlen'] = np.sqrt((((np.abs(temp.Length - temp.meanlen)**2)*temp.Frequency).sum(['Length', 'Sex']))/(temp.n - 1.0))

In [70]:
TS0 = 20.0 * np.log10(temp.Length) - 68.0

In [72]:
temp['TS_lin'] = 10.0 * np.log10((10.0 ** (TS0 / 10.0) * temp.Frequency).sum(['Length', 'Sex']) / (temp.n))

In [75]:
temp['TS_log'] = (TS0 * temp.Frequency).sum(['Length', 'Sex']) / (temp.n)

In [77]:
temp['TS_sd'] = np.sqrt((((np.abs(TS0 - temp.TS_log) ** 2) * temp.Frequency).sum(['Length', 'Sex'])) / (temp.n - 1.0))

In [83]:
temp['nM'] = temp.sel(Sex=1).Frequency.sum('Length')

In [85]:
temp['nF'] = temp.sel(Sex=2).Frequency.sum('Length')

In [87]:
temp

<xarray.Dataset>
Dimensions:    (Haul: 55, Length: 60, Sex: 2)
Coordinates:
  * Haul       (Haul) int64 1 3 7 8 9 10 11 12 13 ... 61 63 65 66 67 69 70 71 73
  * Length     (Length) float64 7.0 9.0 18.0 19.0 20.0 ... 72.0 73.0 74.0 77.0
  * Sex        (Sex) int64 1 2
Data variables:
    Frequency  (Haul, Length, Sex) float64 nan nan nan nan ... nan nan nan nan
    n          (Haul) float64 342.0 535.0 167.0 161.0 ... 187.0 238.0 342.0
    meanlen    (Haul) float64 23.39 20.95 22.24 23.27 ... 43.29 44.53 48.77
    stdlen     (Haul) float64 2.126 1.401 1.248 1.782 ... 3.778 2.557 4.106
    TS_lin     (Haul) float64 -40.59 -41.56 -41.04 ... -35.24 -35.01 -34.21
    TS_log     (Haul) float64 -40.65 -41.6 -41.07 ... -35.31 -35.04 -34.27
    TS_sd      (Haul) float64 0.7475 0.7043 0.489 ... 0.7785 0.5122 0.6952
    nM         (Haul) float64 194.0 97.0 95.0 93.0 ... 133.0 105.0 159.0 144.0
    nF         (Haul) float64 148.0 438.0 72.0 68.0 ... 160.0 82.0 79.0 198.0

In [86]:
temp.nbytes

57256

In [54]:
ds = epro_2019.process_length_data_ds(out, 0)
ds

/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:228: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(


<xarray.Dataset>
Dimensions:     (haul: 55, haul_index: 48)
Coordinates:
  * haul        (haul) int64 1 3 7 8 9 10 11 12 13 ... 63 65 66 67 69 70 71 73
  * haul_index  (haul_index) int64 0 1 2 3 4 5 6 7 8 ... 40 41 42 43 44 45 46 47
Data variables:
    sex         (haul, haul_index) float64 2.0 2.0 2.0 2.0 ... nan nan nan nan
    bio_length  (haul, haul_index) float64 20.0 21.0 22.0 23.0 ... nan nan nan
    frequency   (haul, haul_index) float64 5.0 13.0 38.0 38.0 ... nan nan nan
    n           (haul) float64 342.0 535.0 167.0 161.0 ... 187.0 238.0 342.0
    meanlen     (haul) float64 23.39 20.95 22.24 23.27 ... 43.29 44.53 48.77
    stdlen      (haul) float64 2.126 1.401 1.248 1.782 ... 3.778 2.557 4.106
    TS_lin      (haul) float64 -40.59 -41.56 -41.04 ... -35.24 -35.01 -34.21
    TS_log      (haul) float64 -40.65 -41.6 -41.07 ... -35.31 -35.04 -34.27
    TS_sd       (haul) float64 0.7475 0.7043 0.489 ... 0.7785 0.5122 0.6952
    nM          (haul) float64 194.0 97.0 95.0 93.0 ... 133.0 105.0 159.0 144.0
    nF          (haul) float64 148.0 438.0 72.0 68.0 ... 160.0 82.0 79.0 198.0

In [55]:
# print(sys.getsizeof(df.drop(columns=['Male_ind', 'Female_ind'])))
print(ds.nbytes)

67704


In [6]:
# print(np.all(df['n'].to_numpy() == ds.n.values))
# print(np.allclose(df['meanlen'].to_numpy(), ds.meanlen.values))
# print(np.allclose(df['stdlen'].to_numpy(), ds.stdlen.values))
# print(np.allclose(df['TS_lin'].to_numpy(), ds.TS_lin.values))
# print(np.allclose(df['TS_log'].to_numpy(), ds.TS_log.values))
# print(np.allclose(df['nM'].to_numpy(), ds.nM.values))
# print(np.allclose(df['nF'].to_numpy(), ds.nF.values))

In [7]:
df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_catch_US'])
ds = epro_2019.process_catch_data_ds(df)
ds

<xarray.Dataset>
Dimensions:         (haul: 67, haul_index: 18)
Coordinates:
  * haul            (haul) int64 1 3 4 5 7 8 9 10 11 ... 65 66 67 69 70 71 72 73
  * haul_index      (haul_index) int64 0 1 2 3 4 5 6 7 ... 11 12 13 14 15 16 17
Data variables:
    species_code    (haul, haul_index) object 22500 22602 22610 ... nan nan nan
    species_name    (haul, haul_index) object 'Pacific hake' ... nan
    number_in_haul  (haul, haul_index) object 769.0 355.0 1.0 ... nan nan nan
    weight_in_haul  (haul, haul_index) object 56.25 1.15 0.002 ... nan nan nan
    no_null_length  (haul) int64 12 4 3 1 8 7 6 3 12 3 ... 7 14 17 8 9 12 5 8 6

In [8]:
df = epro_2019.process_catch_data_df(df)
df.head()

,Species_Code,Species_Name,Number_In_Haul,Weight_In_Haul
Haul,,,,
1,22500,Pacific hake,769.0,56.250
1,22602,Northern lampfish,355.0,1.150
1,22610,California headlightfish,1.0,0.002
1,22623,Broadfin lanternfish,1.0,0.006
1,22650,Tarletonbeania sp,1.0,0.004


In [9]:
ds.species_code.sel(haul=73)[0: ds.no_null_length.sel(haul=73).values]

<xarray.DataArray 'species_code' (haul_index: 6)>
array([21110, 21740, 22500, 23010, 23220, 23230], dtype=object)
Coordinates:
    haul        int64 73
  * haul_index  (haul_index) int64 0 1 2 3 4 5

In [10]:
df['Species_Code'].loc[73]

Haul
73    21110
73    21740
73    22500
73    23010
73    23220
73    23230
Name: Species_Code, dtype: int64

In [11]:
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
# ds_us = epro_2019.process_length_data_ds(out, 0)

# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_CAN'])
# ds_can = epro_2019.process_length_data_ds(out, 200)

In [12]:
# ds_us.merge(ds_can)

In [13]:
# df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_trawl_US'])
# df = epro_2019.process_trawl_data(df)

In [14]:
# df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_gear_US'])
# df = epro_2019.process_gear_data(df)

In [15]:
# %%time
# df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_catch_US'])
# df = epro_2019.process_catch_data(df)

In [16]:
# %%time
# df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_specimen_US'])
# df = epro_2019.process_specimen_data(df, 0)
# df.head()

In [17]:
# catch_us_df, length_us_df, trawl_us_df, gear_us_df, specimen_us_df = epro_2019.load_biological_data_us()
# catch_can_df, length_can_df, trawl_can_df, gear_can_df, specimen_can_df = epro_2019.load_biological_data_canada()

In [18]:
# %%time
# df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
# df = epro_2019.process_length_data(df, 0)
# df.head()

In [19]:
# epro_2019.specimen_df.head()

In [20]:
# specimen_ds = epro_2019.specimen_df.to_xarray()

In [21]:
epro_2019.catch_df

,Species_Code,Species_Name,Number_In_Haul,Weight_In_Haul
Haul,,,,
1,22500,Pacific hake,769.0,56.250
1,22602,Northern lampfish,355.0,1.150
1,22610,California headlightfish,1.0,0.002
1,22623,Broadfin lanternfish,1.0,0.006
1,22650,Tarletonbeania sp,1.0,0.004
...,...,...,...,...
222,66770,Glass Shrimp,NaN,2.960
222,22649,Northern flashlightfish,NaN,0.860
222,22602,Northern lampfish,NaN,0.560


In [22]:
# Create catch table with 
# Columns   1-3:   'Trawl number'   'Abundance'     'Biomass' 
epro_2019.final_table_catch = epro_2019.catch_df[epro_2019.catch_df['Species_Code'] == epro_2019.params['species_code_ID']]

In [23]:
# load stratification file 
stratification_index = 1
if stratification_index != 1 and stratification_index != 0:
    raise NotImplementedError(f"stratification_index of {stratification_index} has not been implemented!")
else:
    
    if stratification_index == 1: 
        df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_strata'], sheet_name='Base KS')
        df = df[['Year', 'Cluster name', 'Haul', 'wt']].copy()
        # 1, 2, 4, 5,
        
        # set data types of dataframe
        df = df.astype({'Year': int, 'Cluster name': int, 'Haul': int, 'wt': np.float64})
        
    else:
        df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_strata'], sheet_name='INPFC')
        df = df[['Year', 'INPFC', 'Haul', 'wt']].copy()
        
        # set data types of dataframe
        df = df.astype({'Year': int, 'INPFC': int, 'Haul': int, 'wt': np.float64})
    
    print(df.head())


   Year  Cluster name  Haul   wt
0  2019             1     1  1.0
1  2019             2     3  1.0
2  2019             1     7  1.0
3  2019             1     8  1.0
4  2019             1     9  1.0


In [67]:
# n_len_gender
inter_gear_len = np.intersect1d(epro_2019.gear_df['Haul'].values, epro_2019.length_ds.haul.values)
print(inter_gear_len)
print(epro_2019.gear_df.columns.values)
print(list(epro_2019.length_ds.keys()))
# print(inter_gear_len)
# print(inter_gear_len.shape)
# epro_2019.length_ds.bio_length.sel(haul=inter_gear_len)

# TODO: make this a variable of epro_2019
if stratification_index == 1:
    final_table_trawl = pd.DataFrame(columns=['Haul', 'transect', 'EQlat', 'EQlon', 'Cluster name', 
                                              'ave_dep', 'Tsurf', 'Tdep', 'length', 'sex', 'age', 'wgt', 
                                              'footropeD', 'catch_weight'])
elif stratification_index == 0:
    final_table_trawl = pd.DataFrame(columns=['Haul', 'transect', 'EQlat', 'EQlon', 'INPFC', 
                                              'ave_dep', 'Tsurf', 'Tdep', 'length', 'sex', 'age', 'wgt', 
                                              'footropeD', 'catch_weight'])
else: 
    raise NotImplementedError(f"stratification_index of {stratification_index} has not been implemented!")
    
# for i in inter_gear_len:
    
#     print(i)


[  1   3   7   8   9  10  11  12  13  15  17  19  20  22  24  25  26  27
  28  29  30  31  32  33  35  36  37  38  39  40  42  45  46  47  48  49
  50  51  52  53  54  55  56  57  59  60  61  63  65  66  67  69  70  71
  73 201 203 204 205 206 208]
['Haul' 'Average_Footrope_Depth' 'Surface_Temperature' 'Gear_Temperature'
 'Average_Wireout' 'Transect' 'Net_Height']
['sex', 'bio_length', 'frequency', 'n', 'meanlen', 'stdlen', 'TS_lin', 'TS_log', 'TS_sd', 'nM', 'nF']


In [60]:
# catch_weight

inter_gear_catch = np.intersect1d(epro_2019.gear_df['Haul'].values, epro_2019.catch_df.index.values)

inter_gear_catch

# catch_inter = epro_2019.catch_df.loc[inter_gear_catch]

# catch_weight = catch_inter.loc[catch_inter['Species_Code'] == epro_2019.params['species_code_ID'], 'Weight_In_Haul']

# catch_weight

array([  1,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,  14,  15,
        16,  17,  19,  20,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  69,  70,  71,
        72,  73, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222])

In [48]:
epro_2019.params['species_code_ID']

22500

In [58]:
epro_2019.specimen_df.loc[3].shape

(59, 5)